In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from mlxtend.classifier import StackingCVClassifier
from sklearn import model_selection

# Reading Data

In [3]:
df = pd.read_csv('../../Dataset/NSL_new.csv')

In [4]:
df = df.drop( df[ (df.label != "normal") & (df.label != "neptune") & (df.label != "back") & (df.label != "land") & (df.label != "pod") & (df.label != "smurf") & (df.label != "teardrop") & (df.label != "teardrop") & (df.label != "mailbomb") & (df.label != "apache2") & (df.label != "processtable") & (df.label != "udpstorm") & (df.label != "worm")].index )
print(df.shape)

(113270, 123)


In [5]:
df = df.sample(n=20000)

In [6]:
df.head()

duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
25143          0          0          0     0               0       0    0   
110669         0          0          0     0               0       0    0   
8992           0        146          0     0               0       0    0   
37509          0          0          0     0               0       0    0   
15876          0        212       5790     0               0       0    0   

        num_failed_logins  logged_in  num_compromised  ...  flag_RSTO  \
25143                   0          0                0  ...          0   
110669                  0          0                0  ...          0   
8992                    0          0                0  ...          0   
37509                   0          0                0  ...          0   
15876                   0          1                0  ...          0   

        flag_RSTOS0  flag_RSTR  flag_S0  flag_S1  flag_S2  flag_S3  flag_SF  \
25143             0          0        1        0        0        0        0   
110669            0          0        1        0        0        0        0   
8992              0          0        0        0        0        0        1   
37509             0          0        1        0        0        0        0   
15876             0          0        0        0        0        0        1   

        flag_SH    label  
25143         0  neptune  
110669        0  neptune  
8992          0   normal  
37509         0  neptune  
15876         0   normal  

[5 rows x 123 columns]

In [7]:
X = df.drop(['label'], axis=1)
y = df['label']

In [8]:
scoring_metrics = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average="micro"),
           'recall' : make_scorer(recall_score, average="micro"), 
           'f1_score' : make_scorer(f1_score, average="micro")}

In [9]:
y = y.replace( to_replace =  'normal' , value = 0 )

y = y.replace( to_replace =  'neptune' , value = 1 )
y = y.replace( to_replace =  'back' , value = 1 )
y = y.replace( to_replace =  'land' , value = 1 )
y = y.replace( to_replace =  'pod' , value = 1 )
y = y.replace( to_replace =  'smurf' , value = 1 )
y = y.replace( to_replace =  'teardrop' , value = 1 )
y = y.replace( to_replace =  'mailbomb' , value = 1 )
y = y.replace( to_replace =  'apache2' , value = 1 )
y = y.replace( to_replace =  'processtable' , value = 1 )
y = y.replace( to_replace =  'udpstorm' , value = 1 )
y = y.replace( to_replace =  'worm' , value = 1 )

# adaboost , Random Forest - mlp

In [10]:

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from vecstack import stacking



In [11]:
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score


scoring_metrics = { accuracy_score , precision_score , recall_score , f1_score }

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [23]:
# Create Base Learners
base_learners = [  RandomForestClassifier(n_estimators=90, min_samples_split=0.1 , max_depth=19 ) ,
                    AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)      
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

In [24]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric = accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.98175000]
    fold  1:  [0.97725000]
    fold  2:  [0.98000000]
    fold  3:  [0.98350000]
    ----
    MEAN:     [0.98062500] + [0.00230827]
    FULL:     [0.98062500]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99950000]
    fold  1:  [0.99900000]
    fold  2:  [0.99950000]
    fold  3:  [0.99925000]
    ----
    MEAN:     [0.99931250] + [0.00020729]
    FULL:     [0.99931250]



In [26]:
model = meta_learner.fit(S_train, y_train)


In [27]:
y_pred = meta_learner.predict(S_test)


In [28]:
print('Accuracy score: [%.8f]' % accuracy_score(y_test, y_pred))
print('Precision score: [%.8f]' % precision_score(y_test, y_pred))
print('recall score: [%.8f]' % recall_score(y_test, y_pred))

Accuracy score: [0.99925000]
Precision score: [1.00000000]
recall score: [0.99812851]


In [29]:
stacking1 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) } 

In [34]:
stacking1 = pd.DataFrame(stacking1 , index=[0,1] )

In [36]:
print(stacking1)

   accuracy  Precision    recall
0   0.99925        1.0  0.998129
1   0.99925        1.0  0.998129


In [35]:
stacking1.to_csv('NSL_DOS_Stacking_adaboost-randomForest-mlp.csv')

# adaboost , KNN - mlp

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
# Create Base Learners
base_learners = [KNeighborsClassifier(n_neighbors= 1) ,
                 AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)          
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

In [15]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.99475000]
    fold  1:  [0.99550000]
    fold  2:  [0.99575000]
    fold  3:  [0.99625000]
    ----
    MEAN:     [0.99556250] + [0.00054127]
    FULL:     [0.99556250]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99775000]
    fold  1:  [0.99975000]
    fold  2:  [0.99825000]
    fold  3:  [0.99900000]
    ----
    MEAN:     [0.99868750] + [0.00075777]
    FULL:     [0.99868750]



In [16]:
model = meta_learner.fit(S_train, y_train)

In [17]:
y_pred = meta_learner.predict(S_test)


In [18]:
stacking2 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) } 

In [20]:
stacking2 = pd.DataFrame(stacking2 , index=[0,1] )
print(stacking2)

   accuracy  Precision  recall
0     0.999   0.997523     1.0
1     0.999   0.997523     1.0


In [21]:
stacking2.to_csv('NSL_DOS_Stacking_adaboost-KNN-mlp.csv')

# adaboost , Descision Tree - mlp

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [25]:
from sklearn.tree import tree

# Create Base Learners
base_learners = [tree.DecisionTreeClassifier(max_depth= 13 , min_samples_split = 10 ) ,
                 AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)          
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

In [26]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.99675000]
    fold  1:  [0.99775000]
    fold  2:  [0.99825000]
    fold  3:  [0.99900000]
    ----
    MEAN:     [0.99793750] + [0.00081729]
    FULL:     [0.99793750]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99775000]
    fold  1:  [0.99975000]
    fold  2:  [0.99825000]
    fold  3:  [0.99900000]
    ----
    MEAN:     [0.99868750] + [0.00075777]
    FULL:     [0.99868750]



In [27]:
model = meta_learner.fit(S_train, y_train)

In [28]:
y_pred = meta_learner.predict(S_test)


In [29]:
stacking3 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) } 

In [30]:
stacking3 = pd.DataFrame(stacking3 , index=[0,1] )
print(stacking2)

   accuracy  Precision  recall
0     0.999   0.997523     1.0
1     0.999   0.997523     1.0


In [31]:
stacking3.to_csv('NSL_DOS_Stacking_adaboost-DecisionTree-mlp.csv')

# adaboost , SVM - mlp